#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module  
- 필수오버라이딩  
    * _ _init_ _() : 모델 층 구성 즉, 설계  
    * forward() : 순방향 학습 진행 코드 구현

In [179]:
## 모듈로딩 
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화함수 등등)
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               # 모델 구조 및 정보 관련모듈
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈

In [180]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 피쳐 고정
    * 출력층 - 출력 타겟 고정
    * 은닉층 - 고정

In [181]:
# 모델 설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입력층 :  입력     4개      출력  20개   AF   ReLU
# 은닉층 :  입력    20개      출력 100개   AF   ReLU
# 출력층 :  입력   100개      출력  1개    AF   X,  Sigmoid & Softmax (애넨 분류일때)

In [182]:
# 입력 피쳐가 4개 고정인 정적인 함수수
class Mymodel(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4,20)
        self.hidden_layer=nn.Linear(20, 100)
        self.output_layer=nn.Linear(100,1)

    
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func: 시스템에서 호출되는 함수)
    def forward(self, x):
        print('calling forward()')
        y= self.input_layer(x)   # y= x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)              # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        y=self.hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X20W20+b
        y=F.relu(y)

        return self.output_layer(y)     # 1개 퍼셉트론 : x1W1+x2W2+x3W3+x4W4......X100W100+b


In [183]:
#입력 피쳐가 동적인 모델델
class Mymodel2(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,20)
        self.hidden_layer=nn.Linear(20, 100)
        self.output_layer=nn.Linear(100,1)

    
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func: 시스템에서 호출되는 함수)
    def forward(self, x):
        print('calling forward()')
        y= self.input_layer(x)   # y= x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)              # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        y=self.hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X20W20+b
        y=F.relu(y)

        return self.output_layer(y)     # 1개 퍼셉트론 : x1W1+x2W2+x3W3+x4W4......X100W100+b


In [184]:
#입력 피쳐, 은닉층 퍼셉트론수가 동적인 모델델
class Mymodel3(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature, in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,in_out)
        self.hidden_layer=nn.Linear(in_out, h_out)
        self.output_layer=nn.Linear(h_out,1)

    
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func: 시스템에서 호출되는 함수)
    def forward(self, x):
        print('calling forward()')
        y= self.input_layer(x)   # y= x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)              # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        y=self.hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X20W20+b
        y=F.relu(y)

        return self.output_layer(y)     # 1개 퍼셉트론 : x1W1+x2W2+x3W3+x4W4......X100W100+b


In [185]:
# 은닉층의 개수가 동적인 모델
class Mymodel4(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature, h_range ,in_out, h_out):
        # 부모클래스 생성
        super().__init__()

        self.hidden_layers = nn.ModuleList()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,in_out)
        for _ in range(h_range):
            self.hidden_layers.append(nn.Linear(in_out, h_out))
            in_out = h_out
        self.output_layer=nn.Linear(h_out,1)

    
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func: 시스템에서 호출되는 함수)
    def forward(self, x):
        print('calling forward()')
        y= self.input_layer(x)   # y= x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)              # 0<=y     ---------------> 죽은 relue ==> 해결하기위해 leakyReLu

        for hidden_layer in self.hidden_layers:
            y=hidden_layer(y)   # y= x1W1+x2W2+x3W3+x4W4......X20W20+b
            y=F.relu(y)

        return self.output_layer(y)     # 1개 퍼셉트론 : x1W1+x2W2+x3W3+x4W4......X100W100+b

In [186]:
### 모델 인스턴스 생성
m1 = Mymodel4(3,5,100,70)
m1

Mymodel4(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=100, out_features=70, bias=True)
    (1-4): 4 x Linear(in_features=70, out_features=70, bias=True)
  )
  (input_layer): Linear(in_features=3, out_features=100, bias=True)
  (output_layer): Linear(in_features=70, out_features=1, bias=True)
)

In [187]:
# 모델 파라미터 즉, w와 b 확인
for m in m1.named_parameters(): print(m)

('hidden_layers.0.weight', Parameter containing:
tensor([[-5.0983e-02,  7.0246e-02,  6.4331e-04,  ..., -4.9937e-02,
         -6.7589e-02,  3.7483e-02],
        [-7.9416e-02, -8.3106e-02, -8.9932e-02,  ..., -2.5801e-02,
         -1.4218e-03,  9.8305e-02],
        [ 6.7165e-02, -7.4106e-03,  9.8037e-02,  ..., -3.8047e-02,
          7.0047e-02,  7.0392e-02],
        ...,
        [ 2.5147e-03,  1.6005e-02, -7.4095e-02,  ...,  2.7428e-02,
         -9.5036e-02, -4.7379e-02],
        [-1.0175e-02,  6.7984e-02, -8.7000e-05,  ..., -4.0919e-02,
          9.7483e-02,  7.1616e-02],
        [-1.7923e-03, -8.5366e-02,  3.0353e-02,  ..., -1.0551e-02,
         -2.5179e-02,  2.2959e-02]], requires_grad=True))
('hidden_layers.0.bias', Parameter containing:
tensor([ 0.0512,  0.0184, -0.0734,  0.0909,  0.0447,  0.0348, -0.0207,  0.0914,
         0.0019, -0.0023,  0.0433, -0.0067,  0.0177,  0.0003,  0.0237,  0.0021,
        -0.0635, -0.0293,  0.0873,  0.0653, -0.0105,  0.0080, -0.0652,  0.0730,
        -0.

In [188]:
# 학습 진행 ==> 모델인스턴스명(데이터)
# 임의의 데이터
dataTS=torch.FloatTensor([[1,3,5], [2,4,6]])  # 입력층 4개라했으니까 4개 넣어줘야함함
targetTS=torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)

print(pre_y)

calling forward()
tensor([[0.0325],
        [0.0310]], grad_fn=<AddmmBackward0>)
